In [2]:
import numpy as np
import cv2

import PIL.Image as Image
#import os

import matplotlib.pylab as plt

# import tensorflow as tf
#import tensorflow_hub as tensorflow_hub

from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.models import Sequential

import pathlib
import dlib

from sklearn.metrics import euclidean_distances
from tensorflow.keras.optimizers import Adam

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../input/shape-predictor/shape_predictor_68_face_landmarks.dat')

In [4]:
confused_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Engaged/confused"
engaged_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Engaged/engaged"
frustrated_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Engaged/frustrated"

Lookingaway_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Not engaged/Looking Away"
bored_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Not engaged/bored"
drowsy_dataset_dir = "../input/studentengagement/Student-engagement-dataset/Not engaged/drowsy"


confused_dataset_dir = pathlib.Path(confused_dataset_dir)
print(confused_dataset_dir)
engaged_dataset_dir = pathlib.Path(engaged_dataset_dir)
print(engaged_dataset_dir)
frustrated_dataset_dir = pathlib.Path(frustrated_dataset_dir)
print(frustrated_dataset_dir)
Lookingaway_dataset_dir = pathlib.Path(Lookingaway_dataset_dir)
print(Lookingaway_dataset_dir)
bored_dataset_dir = pathlib.Path(bored_dataset_dir)
print(bored_dataset_dir)
drowsy_dataset_dir = pathlib.Path(drowsy_dataset_dir)
print(drowsy_dataset_dir)

In [5]:
list(confused_dataset_dir.glob('*'))[:5]

In [6]:
image_count = len(list(confused_dataset_dir.glob('*')))
image_count

In [7]:
confused = list(confused_dataset_dir.glob("*"))
Image.open(str(confused[5])) 

In [8]:
image_dict = {
    'confused': list(confused_dataset_dir.glob('*')),
    'engaged' : list(engaged_dataset_dir.glob('*')),
    'frustrated': list(frustrated_dataset_dir.glob('*')),
    'Lookingaway' : list(Lookingaway_dataset_dir.glob('*')),
    'bored': list(bored_dataset_dir.glob('*')),
    'drowsy' : list(drowsy_dataset_dir.glob('*')),
}

In [9]:
labels_dict = {
    'confused': 0,
    'engaged' : 1,
    'frustrated': 2,
    'Lookingaway' : 3,
    'bored': 4,
    'drowsy' : 5,
}

labels_class = ['confused','engaged','frustrated','Lookingaway','bored','drowsy',]

In [12]:
X = []
y = []

for label, images in image_dict.items():
  for image in images:
    img = cv2.imread(str(image))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    if len(rects) == 0:
      continue
    shape = predictor(gray, rects[0])

    shape_np = np.zeros((68, 2), dtype="int")

    for i in range(0, 68):
        shape_np[i] = (shape.part(i).x, shape.part(i).y)
    shape = shape_np

    eucl_dist = euclidean_distances(shape, shape)

    X.append(eucl_dist)
    y.append(labels_dict[label])

In [13]:
X = np.array(X)
y = np.array(y)

In [33]:
X.shape
#y.shape

In [34]:
X_reshaped = X.reshape(3,-1)

In [35]:
np.savetxt("xdata.csv", X_reshaped, delimiter=",")
np.savetxt("ydata.csv", y, delimiter=",")

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [37]:
X_train.shape

In [48]:
import tensorflow as tf
X_train = tf.expand_dims(X_train, axis=-1)
X_test = tf.expand_dims(X_test, axis=-1)

In [50]:
model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(68, 68)),  # input layer (1)
#     keras.layers.Dense(512, activation='sigmoid'),  # hidden layer (2)
#     keras.layers.Dense(512, activation='sigmoid'),  # hidden layer (3)
#     keras.layers.Dense(512, activation='sigmoid'),  # hidden layer (4)
#     keras.layers.Dense(512, activation='sigmoid'),  # hidden layer (5)
#     keras.layers.Dense(6, activation='softmax') # output layer (6)
    
    keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(68,68,1)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(68,68,1)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(6,activation='softmax')
])

In [53]:
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt,
             loss='sparse_categorical_crossentropy',
             metrics=["accuracy"])

model.summary()

In [54]:
history = model.fit(X_train,y_train,epochs = 10 , validation_data = (X_test, y_test))

In [58]:
model.save('model_landmark.h5')

In [57]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()